In [20]:
import pandas as pd
import numpy as np

import scipy.io as sc
import scipy.stats
from scipy.fft import fft, fftfreq
from scipy.fftpack import fft, ifft 
from os import listdir
import os.path
import shutil 
from IPython.display import clear_output

import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
 success = True
while success == True:
    try:
        shutil.rmtree('corrected')
        success = False
    except:
        success = False

In [38]:
success = True
while success == True:
    try: 
        list_of_folders = listdir("cutting_tests_processed")
        for folder in list_of_folders:
            os.makedirs(f'corrected/{folder}')
        success = False
    except:
        success = False

In [39]:
def main_():
    '''
    changes the split files first letter to match it's correct classification to
    the presence of chatter according to the users input and saves the file with
    the corect name to the folder "corrected" with the same folders as 
    split_cutting_tests_processed.
    '''
    folders = listdir('split_cutting_tests_processed')
    print(folders)
    # folder = ''
    path_split = 'split_cutting_tests_processed'
    path_corrected = 'corrected'

    for folder in folders:
        list_of_files = listdir(f'{path_split}/{folder}') 
        for file in list_of_files:
            if file[0] != 's' and file[-5] != 'd':
                print(f'Folder: {folder}') 
                print(f'File: {file}')
                print('')

                plot = pd.read_csv(f'{path_split}/{folder}/{file}')

                # data = sc.loadmat(f'cutting_tests_processed/{folder}/{file[0:9]}.mat')
                # lim = pd.DataFrame.from_dict(data['tsDS'][:,:])
                # lim.rename({0: 't', 1: 'y'}, axis=1, inplace=True)
                # limit = np.fft.fft(lim['y']).max()

                fft = np.fft.fft(plot['y'])
                fftfreq = np.fft.fftfreq(len(plot['t']), plot['t'][1] - plot['t'][0])

                # ploting
                fig, ax = plt.subplots(1,2, figsize = (10,4))
                # plt.ylim(0,limit)
                ax[0].plot(plot['t'], plot['y'])
                ax[1].plot(fftfreq[10:int(len(fft)/2)], np.abs(fft[10:int(len(fft)/2)])/len(fft), c = 'k')
                fig.suptitle(file)
                plt.show()

                # classifing 
                print("c = has chatter \n s = no chatter \n or \"stop\"")
                presence = input('')
                if presence == 'stop':
                    break

                # saving files
                shutil.copyfile(f'{path_split}/{folder}/{file}', f'{path_corrected}/{folder}/{presence}{file[1:]}')
                shutil.copyfile(f'{path_split}/{folder}/{file[:-4]}_noise_added.csv', f'{path_corrected}/{folder}/{presence}{file[1:-4]}_noise_added.csv')

                clear_output()


In [40]:
main_()

# Creating a new data frame with math features 

Defining the functions used in the creation of the dataframe as to keep the formulas more organized and easy to read. 

In [1]:
def thd(data_): # true harmonic distortion
    sq_sum = 0.0
    for r in range(len(data_)):
        sq_sum = sq_sum + data_[r]**2
    sq_harmonics = sq_sum - max(data_)**2
    return 100*sq_harmonics**0.5 / max(data_)

def rms(data_):
    return np.sqrt(np.mean(data_**2))

def amp(data_):
    return max(data_) - min(data_)

def SRA(data_):
    return np.mean(np.sqrt(np.abs(data_)))**2

def crest(data_):
    return max(np.abs(data_))/np.sqrt(np.mean(data_**2))

def impulse(data_):
    return max(np.abs(data_))/np.mean(abs(data_))

def margin(data_):
    return max(abs(data_))/np.mean(np.sqrt(np.abs(data_)))**2

def freq_max(data_):
    # sorted(np.array(scipy.fftpack.fft(file['y'].values)))[-2]
    # print(data_)
    print(data_['fftfreq'][data_['fft'] == data_['fft'].max()].values[0])

def freq_sec_highest(data_):
    # sorted(np.array(scipy.fftpack.fft(file['fftfreq'].values)))[-3]
    data_sorted = data_.sort_values(by = ['fft'], ascending = False)
    return data_sorted['fftfreq'][1]

def freq_mean(data_):
    return data_['fftfreq'].mean()

# def freq_pico(data_):
#     # sorted(np.array(scipy.fftpack.fft(file['y'].values)))[-2]
#     return data_['fftfreq'].max()

#         cols['freq_max'].append(scipy.fftpack.fft(file['y'].values).max()) # https://gist.github.com/endolith/255291
#         cols['freq_sec_highest'].append(sorted(np.array(scipy.fftpack.fft(file['y'].values)))[-2]) 
#         cols['freq_mean'].append(scipy.fftpack.fft(file['y'].values).mean()) 


In [ ]:
''' Creating the data frame of math equations outputs.
'''
cols = {'Experiment': [],
        'chatter_presence': [],
        'med': [],
        'std': [],
        'rms': [],
        'curtose': [],
        'distortion': [],
        'amplitude': [],
        'sra': [],
        'crest': [],
        'impulse': [],
        'margin': [],
        'max': [],
        'freq_max': [],
        'freq_sec_highest': [],
        'freq_mean': []}

list_of_folders = listdir("corrected")
for folder in list_of_folders:
    list_of_files = listdir(f'corrected/{folder}')
    for file in list_of_files:
        cols['Experiment'].append(f'{file[:-4]}')
        cols['chatter_presence'].append(f'{file[:1]}')
        
        # print(file)
        file = pd.read_csv(f'corrected/{folder}/{file}', index_col=0)
                
        cols['med'].append(file['y'].mean())
        cols['std'].append(file['y'].std())
        cols['rms'].append(rms(file['y']))
        
        cols['curtose'].append(scipy.stats.kurtosis(file['y']))
        cols['distortion'].append(thd(file['y']))
        cols['amplitude'].append(amp(file['y']))
        cols['sra'].append(SRA(file['y']))
        
        cols['crest'].append(crest(file['y']))
        cols['impulse'].append(impulse(file['y']))
        cols['margin'].append(margin(file['y']))
        cols['max'].append(file['y'].max())
        
        cols['freq_max'].append(freq_max(file)) # https://gist.github.com/endolith/255291
        cols['freq_sec_highest'].append(freq_sec_highest(file)) 
        cols['freq_mean'].append(freq_mean(file))
        clear_output()

dataframe = pd.DataFrame(cols)

In [ ]:
dataframe

In [1]:
dataframe.to_csv('math_outputs.csv')

NameError: name 'dataframe' is not defined